In [1]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from   torchtext.vocab import GloVe
from   torchtext.legacy.data import Field
from   torch.optim import lr_scheduler

##### 构造数据集

In [2]:
# 创建字段
TEXT                 = torchtext.legacy.data.Field(lower=True,fix_length=200,batch_first=True)
LABEL                = torchtext.legacy.data.Field(sequential=False)
# 加载数据
train,test           = torchtext.legacy.datasets.IMDB.splits(TEXT,LABEL)
# 构建词表  使用GLoVe库的内置词向量 将词表映射到100维的向量空间中
TEXT.build_vocab(train,vectors=GloVe(name="6B",dim=100),max_size=20000,min_freq=10)
LABEL.build_vocab(train)
# 创建DataLoader
train_iter,test_iter = torchtext.legacy.data.BucketIterator.splits((train,test),batch_size=16)

##### 创建模型

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 将单词做映射 参数为: 输入词表长度 以及 用多长的向量表示一个的单词
        self.em  = nn.Embedding(len(TEXT.vocab.stoi),100) # batch*200单词 <--> batch*200*100向量
        self.fc1 = nn.Linear(200*100,1024)                # 
        self.fc2 = nn.Linear(1024,3)                      # 定义为3分类模型
    def forward(self,x):
        x = self.em(x)                                    # 文本的词向量表示
        x = x.view(x.size(0),-1)                          # 将数据扁平化
        x = self.fc1(x)
        x = F.relu(x)                                     # 激活
        x = self.fc2(x)                                   # 输出值生成
        return x

In [4]:
model                = Net()
model                = model.to("cuda")
loss_fn              = nn.CrossEntropyLoss()
optimizer            = torch.optim.Adam(model.parameters())
exp_lr_scheduler     = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
epochs               = 10

In [5]:
''' 使用已经训练好的词向量 '''
model.em.weight.data          = TEXT.vocab.vectors
model.em.weight.requires_grad = False
''' 将model.em.weight从model.parameters()优化器中排除 '''
optimizer  =  torch.optim.Adam(
        [param for param in model.parameters() if param.requires_grad==True],
        lr = 0.001
)
model      = model.cuda()

##### 训练模型

In [6]:
def fit(epoch, model, trainloader, testloader):
    correct      = 0
    total        = 0
    running_loss = 0
    
    model.train()
    for b in trainloader:
        x, y     = b.text, b.label
        if torch.cuda.is_available():
            x, y = b.text.to('cuda'), b.label.to('cuda')
        y_pred   = model(x)
        loss     = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred       = torch.argmax(y_pred, dim=1)
            correct      += (y_pred == y).sum().item()
            total        += y.size(0)
            running_loss += loss.item()
    exp_lr_scheduler.step()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc  = correct / total
        
        
    test_correct      = 0
    test_total        = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y     = b.text, b.label
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred             = model(x)
            loss               = loss_fn(y_pred, y)
            y_pred             = torch.argmax(y_pred, dim=1)
            test_correct      += (y_pred == y).sum().item()
            test_total        += y.size(0)
            test_running_loss += loss.item()
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc  = test_correct / test_total  
    print('epoch: ', epoch, '\t loss： ', round(epoch_loss, 3),'\t accuracy:', round(epoch_acc, 3),'\t test_loss： ', round(epoch_test_loss, 3),'\t test_accuracy:', round(epoch_test_acc, 3))
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

In [7]:
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch, model,train_iter,test_iter)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

D:\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch:  0 	 loss：  0.041 	 accuracy: 0.656 	 test_loss：  0.036 	 test_accuracy: 0.694
epoch:  1 	 loss：  0.032 	 accuracy: 0.753 	 test_loss：  0.038 	 test_accuracy: 0.698
epoch:  2 	 loss：  0.025 	 accuracy: 0.819 	 test_loss：  0.04 	 test_accuracy: 0.696
epoch:  3 	 loss：  0.019 	 accuracy: 0.871 	 test_loss：  0.045 	 test_accuracy: 0.688
epoch:  4 	 loss：  0.011 	 accuracy: 0.925 	 test_loss：  0.065 	 test_accuracy: 0.692
epoch:  5 	 loss：  0.007 	 accuracy: 0.956 	 test_loss：  0.075 	 test_accuracy: 0.692
epoch:  6 	 loss：  0.005 	 accuracy: 0.969 	 test_loss：  0.104 	 test_accuracy: 0.674
epoch:  7 	 loss：  0.004 	 accuracy: 0.975 	 test_loss：  0.113 	 test_accuracy: 0.677
epoch:  8 	 loss：  0.004 	 accuracy: 0.977 	 test_loss：  0.115 	 test_accuracy: 0.683
epoch:  9 	 loss：  0.003 	 accuracy: 0.981 	 test_loss：  0.108 	 test_accuracy: 0.683
